# Cooking Assistant

## openai
The library that will make it easier for us to connect directly to the openAI API.

## os and dotenv
The libraries that allow me to upload this directly to github without exposing my API credentials.

In [15]:
import os
import openai
from dotenv import load_dotenv

In [ ]:
load_dotenv()

## Connecting to the openAI API

In [13]:
openai.api_key = os.getenv("OPEN_API_KEY")

## generate_recipe
This function formats the prompt that is sent to GPT3 (text-davinci-003). It takes variables that will eventually be available inputs on a webpage.

In [14]:
def generate_recipe(cooking_skill, place_city, preparation_time):
    prompt = f"Please provide a detailed cooking recipe with ingredients and step-by-step directions, suitable for someone with {cooking_skill}, originating from {place_city}, and taking {preparation_time} time to prepare and cook.\nPlease provide this in JSON format with the following variables:\nname, description, ingredients, directions"

    response = openai.Completion.create(
        engine='text-davinci-003',
        prompt=prompt,
        max_tokens=1000,
        n=1,
        temperature=0,
        top_p=0.5,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].text.strip()

In [10]:
cooking_skill = "advanced"
place_city = "Dallas"
preparation_time = "long"

## recipe_json
Here we will be executing the function by passing our 3 variables into the prompt, which is then sent to openAPI

In [11]:
recipe_json = generate_recipe(cooking_skill, place_city, preparation_time)

## JSON Structure
This JSON response will be used to fill in our webpage in an digestible format.

In [12]:
print(recipe_json)

{
  "name": "Dallas-Style BBQ Brisket",
  "description": "This recipe is a classic Dallas-style BBQ brisket that takes a long time to prepare and cook, but is worth the wait!",
  "ingredients": [
    "5-7 lb brisket",
    "1/4 cup of your favorite BBQ rub",
    "1/4 cup of your favorite BBQ sauce",
    "1/4 cup of beef broth",
    "1/4 cup of apple cider vinegar",
    "1/4 cup of Worcestershire sauce",
    "1/4 cup of brown sugar",
    "1/4 cup of honey",
    "1/4 cup of molasses",
    "1/4 cup of tomato paste",
    "1/4 cup of garlic powder",
    "1/4 cup of onion powder",
    "1/4 cup of chili powder",
    "1/4 cup of smoked paprika",
    "1/4 cup of cumin",
    "1/4 cup of black pepper",
    "1/4 cup of cayenne pepper"
  ],
  "directions": [
    "1. Preheat your oven to 250 degrees F.",
    "2. Trim the fat from the brisket and rub the BBQ rub all over the brisket.",
    "3. Place the brisket in a roasting pan and cover with aluminum foil.",
    "4. Bake in the preheated oven for 3-